In [2]:
# Tests
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.parse import urlparse
from requests_html import HTMLSession

# sample youtube video url
video_url = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'
# init an HTML Session
session = HTMLSession()
# get the html content
response = session.get(video_url)
# execute Java-script
response.html.render(sleep=1)
# create bs object to parse HTML
soup = BeautifulSoup(response.html.html, "html.parser")

urls = ['https://www.youtube.com/watch?v=dQw4w9WgXcQ']
response = requests.get(urls[0]) # send GET request to url
#soup = BeautifulSoup(response.text, 'html.parser') # parse the text given by request
vidTitle = soup.title.text
# print(soup.prettify())
soup.find("meta", itemprop="name")["content"] # title
soup.find("meta", itemprop="interactionCount")['content'] # view count
soup.find("meta", itemprop="description")['content'] # description
soup.find("meta", itemprop="datePublished")['content'] # video publish date
soup.find("span", {"class": "ytp-time-duration"}) # video length
', '.join([ meta.attrs.get("content") for meta in soup.find_all("meta", {"property": "og:video:tag"}) ]) # video tags

# channel details
channel_tag = soup.find("yt-formatted-string", {"class": "ytd-channel-name"}).find("a")
# channel name
channel_name = channel_tag.text
# channel URL
channel_url = f"https://www.youtube.com{channel_tag['href']}"
# number of subscribers as str
channel_subscribers = soup.find("yt-formatted-string", {"id": "owner-sub-count"}).text.strip()
result = {'name': channel_name, 'url': channel_url, 'subscribers': channel_subscribers}


RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.

In [ ]:
# covcom fctn

from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.parse import urlparse
import numpy as np

urls = ['https://www.youtube.com/watch?v=dQw4w9WgXcQ']

#provider_details = pd.DataFrame(columns=['company_name','company_link', 'company_number', 'company_email'])

for url in urls:    
    # scrape page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    providers_table = soup.find('table', {'class', 'govuk-table'}).find('tbody')

    url_provider_details = pd.DataFrame(columns=['company_name', 'company_link', 'company_number', 'company_email'])
    for row in providers_table.find_all('tr'):
        # test provider saved in cell with id 'provider'
        provider = row.find(id='provider').find('a')
        name = provider.get_text().rstrip().lower()
        link = provider['href']
    
        # test provider number and email saved in only cell(s) with no id
        number_email = row.find_all('td', id=None)
        
        # remove web archive prefix from wayback machine
        link = link[43:] if 'web.archive.org' in link else link

        # old format stores number and email in separate cells
        # new format stores number and email in same cell
        if len(number_email) == 1:
            number_email = number_email[0].find_all('a')
        number = str(number_email[0].get_text())
        email = number_email[1].get_text()
        
        # apply standard format to numbers, emails and links
        if number and len(number) >= 10:
            number = ' '.join(number.rstrip().split())
            number = phonenumbers.format_number(phonenumbers.parse(number, 'GB'), phonenumbers.PhoneNumberFormat.INTERNATIONAL)
        else:
            number = np.nan 

        if email:
            email = str(email).lower()

        if link:
            link = urlparse(link.lower()).netloc

        url_provider_details = url_provider_details.append({
            'company_name': name,
            'company_link': link,
            'company_number': number,
            'company_email': email
        }, ignore_index=True)

    provider_details = pd.merge(
        provider_details, 
        url_provider_details, 
        how="outer", 
        on=['company_name','company_link','company_number','company_email']
    )

provider_details.to_csv('datasets/provider_details.csv')